# Neo-Hookean compressible solid material

Neo-Hooke compressible material is a hyperelastic material with strain energy density (for $d=3$) given as
$$
W(I_1, I_3) = C_1 (I_1 - 3 - 2 \log I_3) + D_1 (I_3 - 1)^2
$$
where
$$
\begin{aligned}
I_3 &= \det \bm{F},\\
I_1 &= \text{Tr} (\bm F^T \bm F),\\
\bm F &= \bm I + \nabla \bm u.
\end{aligned}
$$

Material parameters are chosen as $C_1 = \mu / 2, \, D_1 = \lambda / 2$ for consistency with linear Hooke elasticity, $\mu$ is the second Lamé parameter (shear modulus) and $\lambda$ is first Lamé parameter.

We can formulate the following (unconstrained) optimization problem: find $\bm u \in [H^1_0(\Omega)]^3$ that solves
$$
\min\limits_{\bm u \in [H^1_0(\Omega)]^3} \left\{ \int_\Omega W(I_1, I_3) \, \mathrm dx - \int_{\Gamma_N} \bm t \cdot \bm u \, \mathrm ds \right\}.
$$

This functional is not convex in $\bm F$ (since $\det \bm F$ is not).

In [40]:
import gmsh

if not gmsh.is_initialized():
    gmsh.initialize()
gmsh.clear()
gmsh.open("model.step")

mesh_size = 4

gmsh.option.setNumber("Mesh.MeshSizeMax", mesh_size)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(3, [1], 1)
gmsh.model.addPhysicalGroup(2, [1], 2)
gmsh.model.addPhysicalGroup(2, [4], 3)
gmsh.model.mesh.generate()


Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Reading 'model.step'...
Info    :  - Label 'Shapes/Body' (3D)
Info    :  - Color (0.447059, 0.47451, 0.501961) (3D & Surfaces)
Info    : Done reading 'model.step'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Line)
Info    : [ 30%] Meshing curve 8 (Line)
Info    : [ 40%] Meshing curve 9 (Line)
Info    : [ 40%] Meshing curve 10 (Line)
Info    : [ 50%] Meshing curve 11 (Line)
Info    : [ 50%] Meshing curve 12 (Line)
Info    : [ 60%] Meshing curve 13 (Line)
Info    : [ 60%] Meshing curve 14 (Line)
Info    : [ 60%] Meshing curve 15 (Line)
Info    : [ 70%] Meshing curve 16 (Line)
Info    : [ 70%] Meshing curve 17 (Line)
Info    : [ 80%] Meshing cu

In [41]:
import dolfinx
from mpi4py import MPI
mesh, cell_tags, facet_tags = dolfinx.io.gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, rank=0, gdim=3)
fixed_facets = facet_tags.indices[facet_tags.values == 2]
loaded_facets = facet_tags.indices[facet_tags.values == 3]

print(f"Number of fixed facets: {len(fixed_facets)}")
print(f"Number of loaded facets: {len(loaded_facets)}")

Number of fixed facets: 18
Number of loaded facets: 14


In [42]:
import pyvista as pv
import os
os.environ["LIBGL_ALWAYS_SOFTWARE"] = "1"
pv.set_jupyter_backend("html")
pv.start_xvfb()

pv.global_theme.window_size = [600, 500]

In [43]:
cells, types, x = dolfinx.plot.vtk_mesh(mesh)
grid = pv.UnstructuredGrid(cells, types, x)

plotter = pv.Plotter()
plotter.add_mesh(grid, show_edges=True, lighting=False)
plotter.camera_position = "xy"
plotter.camera.elevation = 25
plotter.enable_parallel_projection()
plotter.show_axes()
plotter.show_grid()
plotter.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

The choice of discrete space for the displacement we make is $[P_2]^3 \subset [H^1(\Omega)]^3$, i.e. a vector-valued degree 2 Lagrange space, see [defelement.org](https://defelement.org/elements/examples/tetrahedron-vector-lagrange-2.html),

<div align="center"><img src="images/vp2.png" alt="lagrange-p1" width="600"/></div>

In [44]:
V = dolfinx.fem.functionspace(mesh, ("P", 2, (3, )))
print(f"Number of degrees of freedom: {V.dofmap.index_map.size_global} x {V.dofmap.index_map_bs}")

Number of degrees of freedom: 1540 x 3


In [45]:
import ufl
import numpy as np

nu = 0.49
E = 3.0  # MPa = N/mm^2, natural rubber

mu = E / (2 * (1 + nu))
lmbda = E * nu / ((1 + nu) * (1 - 2 * nu))

C1 = mu / 2
D1 = lmbda / 2


def W(I1, I3):
    return C1*(I1 - 3 - 2*ufl.ln(I3)) + D1*(I3 - 1)**2


u0 = dolfinx.fem.Function(V)

F = ufl.Identity(3) + ufl.grad(u0)
I1 = ufl.tr(F.T * F)
I3 = ufl.det(F)

x = ufl.SpatialCoordinate(mesh)
t = dolfinx.fem.Constant(mesh, np.array([0.0, 0.0, 0.0]) * 1e-6)

ds = ufl.Measure("ds", domain=mesh, subdomain_data=facet_tags)
W_total = W(I1, I3) * ufl.dx(metadata={"quadrature_degree": 4}) - ufl.inner(t, u0) * ds(3)

In [46]:
boundary_dofs = dolfinx.fem.locate_dofs_topological(V, entity_dim=2, entities=fixed_facets)
bc = dolfinx.fem.dirichletbc(dolfinx.fem.Constant(mesh, [0.0, 0.0, 0.0]), dofs=boundary_dofs, V=V)

loaded_dofs = dolfinx.fem.locate_dofs_topological(V, entity_dim=2, entities=loaded_facets)
bc1 = dolfinx.fem.dirichletbc(dolfinx.fem.Constant(mesh, [0.0, 0.0, 0.0]), dofs=loaded_dofs, V=V)

In [47]:
import dolfinx.nls.petsc
from petsc4py import PETSc

R = ufl.derivative(W_total, u0)
problem = dolfinx.fem.petsc.NonlinearProblem(R, u0, bcs=[bc, bc1])
solver = dolfinx.nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem)

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "cholesky"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

dolfinx.log.set_log_level(dolfinx.log.LogLevel.WARNING)
for i in range(20):
    n, converged = solver.solve(u0)
    print(f"Number of Newton iterations: {n}")
    print(f"Total displacement: {bc1.g.value[2]} mm")
    bc1.g.value[2] += 1

Number of Newton iterations: 0
Total displacement: 0.0 mm
Number of Newton iterations: 4
Total displacement: 1.0 mm
Number of Newton iterations: 4
Total displacement: 2.0 mm
Number of Newton iterations: 4
Total displacement: 3.0 mm
Number of Newton iterations: 4
Total displacement: 4.0 mm
Number of Newton iterations: 4
Total displacement: 5.0 mm
Number of Newton iterations: 4
Total displacement: 6.0 mm
Number of Newton iterations: 4
Total displacement: 7.0 mm
Number of Newton iterations: 4
Total displacement: 8.0 mm
Number of Newton iterations: 4
Total displacement: 9.0 mm
Number of Newton iterations: 4
Total displacement: 10.0 mm
Number of Newton iterations: 4
Total displacement: 11.0 mm
Number of Newton iterations: 4
Total displacement: 12.0 mm
Number of Newton iterations: 4
Total displacement: 13.0 mm
Number of Newton iterations: 4
Total displacement: 14.0 mm
Number of Newton iterations: 4
Total displacement: 15.0 mm
Number of Newton iterations: 4
Total displacement: 16.0 mm
Number 

In [48]:
def plot(u, title):
    cells, types, x = dolfinx.plot.vtk_mesh(u.function_space)
    grid = pv.UnstructuredGrid(cells, types, x)

    if not grid.get_cell(0).is_linear:
        levels = 4
    else:
        levels = 0

    grid.point_data["u"] = u.x.array.reshape(-1, 3)
    grid.set_active_scalars("u")
    grid_warped = grid.warp_by_vector("u")

    plotter = pv.Plotter()
    plotter.add_mesh(grid_warped.extract_surface(nonlinear_subdivision=levels),
                     ambient=0.5,
                     specular=0.5,
                     specular_power=20,
                     n_colors=10,
                     scalar_bar_args={"title": title})
    plotter.add_mesh(grid_warped.separate_cells().extract_surface(nonlinear_subdivision=levels).extract_feature_edges(),
                     style="wireframe",
                     color="black")

    plotter.camera_position = "xy"
    plotter.enable_parallel_projection()
    plotter.show_axes()
    plotter.show()

In [49]:
plot(u0, "Displacement [mm]")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…